In [1]:
import numpy as np
import sklearn
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas

In [2]:
data_train = pandas.read_csv('salary-train.csv') 
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [3]:
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_train['FullDescription'] = [text.lower() for text in data_train['FullDescription']]

In [4]:
vectorizer = TfidfVectorizer(min_df = 5)
feaut = vectorizer.fit_transform(data_train['FullDescription'])
feaut

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [5]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train_categ 

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [6]:
from scipy.sparse import hstack

In [7]:
ofmatrix = hstack([feaut, X_train_categ])
ofmatrix

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [8]:
lin_reg = Ridge(alpha=1,random_state=241)
lin_reg = lin_reg.fit(ofmatrix, data_train['SalaryNormalized'])

In [9]:
data_test = pandas.read_csv('salary-test-mini.csv')
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [10]:
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = [text.lower() for text in data_test['FullDescription']]
feaut_test = vectorizer.transform(data_test['FullDescription'])
from sklearn.feature_extraction import DictVectorizer
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([feaut_test, X_test_categ])
X_test

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [11]:
lin_reg.predict(X_test)

array([ 56555.61500155,  37188.32442618])